This is a template to reconstruct holography images.

# Import libraries

Start by importing standard python libraries and the self written libraries $\mathtt{fth-reconstruction.py}$, $\mathtt{reconstruct.py}$ and $\mathtt{cameras.py}$. They have to be in the same folder or the parent directory.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import h5py

# import self written libraries. These have to be in the same folder or the parent directory.
import sys, os
sys.path.append('./library/')
import fth_reconstruction as fth
import reconstruct as reco
import matplotlib as mpl
import ipywidgets as widgets

In [2]:
# interactive plotting
%matplotlib widget

# Load images 

Start by loading the images. 
1. Specify the directory of the data and the number of the positive and negative helicity image.
2. Load the data.
3. Create the difference hologram by subtracting the topography image (pos + neg) from the positive image. Set the auto_factor to *True*, otherwise the default factor of 0.5 will be used.

In [3]:
INSTRUMENT_KEYS = {
    'mono': 'scan/instrument/mono/energy',
    'undu_shift': 'scan/instrument/undulator/collection/shift',
    'ringcurrent': 'scan/instrument/petra_iii/current',
    'exitslit': 'scan/instrument/exit_slit/slit/y_gap'
    }


def load_scanfile(fname):
    scandata = xr.Dataset()
    dimcount = 0
    with h5py.File(fname, 'r') as f:
        for k in f['scan/data'].keys():
            data = f['scan/data/' + k][()]
            dims = ['index']
            ndim = len(data.shape)
            for i in range(ndim - 1):
                dims.append(f'dim_{dimcount}')
                dimcount += 1
            scandata[k] = (dims, data)
        for k, v in INTRUMENT_KEYS.items():
            scandata[k] = ('instrument', f[v][()])
    return scandata



In [4]:
experimental_setup = {'ccd_dist': 18e-2, 'energy': 779.5, 'px_size' : 13.5e-6}

In [8]:
ls '/home/mschndr/MaxP04_2103_commissioning/raw/'

comm_00001.h5              test_00073.fio  test_00102.nxs  test_00134.fio
comm_00002.h5              test_00073.nxs  test_00103.fio  test_00134.nxs
comm_00003.h5              test_00074.fio  test_00103.nxs  test_00135.fio
comm_00004.h5              test_00074.nxs  test_00104.fio  test_00135.nxs
comm_00005.h5              test_00075.fio  test_00104.nxs  test_00136.fio
comm_00006.h5              test_00075.nxs  test_00105.fio  test_00136.nxs
comm_00007.h5              test_00076.fio  test_00105.nxs  test_00137.fio
comm_00008.h5              test_00076.nxs  test_00106.fio  test_00137.nxs
commissioning_01_00001.h5  test_00077.fio  test_00106.nxs  test_00138.fio
test_00001.h5              test_00077.nxs  test_00107.fio  test_00138.nxs
test_00002.h5              test_00078.fio  test_00107.nxs  test_00139.fio
test_00048.fio             test_00078.nxs  test_00108.fio  test_00139.nxs
test_00048.nxs             test_00079.fio  test_00108.nxs  test_00140.fio
test_00049.fio             test_00079.

In [12]:
folder = '/home/mschndr/MaxP04_2103_commissioning/raw/'
fname = 'test_%05d.nxs'

d1 = load_scanfile(folder + fname % 148)
d2 = load_scanfile(folder + fname % 150)

In [6]:
d1 = load_scanfile(folder + fname % 145)
d2 = load_scanfile(folder + fname % 147)

pos = d1.ccd.sum('index').values
neg = d2.ccd.sum('index').values

In [27]:
holo_sum = pos + neg
holo_sum -= np.percentile(holo_sum, 1)
holo_diff = pos - neg

In [50]:
vmin, vmax = np.percentile(holo_sum, [1, 99.9])
lognorm = mpl.colors.LogNorm(5e3, 5e5)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(holo_sum, cmap='YlGnBu_r', vmin=vmin, vmax=vmax)
# ax.imshow(holo_sum, cmap='YlGnBu_r', norm=lognorm)

ax.set_axis_off()
ax.set_xlim(760, 1400)
ax.set_ylim(835, 1475)
plt.tight_layout(pad=.1)
fig.savefig('210323_holo_sum.png', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
vmin, vmax = np.percentile(holo_sum, [1, 99.9])
lognorm = mpl.colors.LogNorm(5e3, 5e5)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(holo_diff, cmap='coolwarm', vmin=-.5e5, vmax=.5e5)
# ax.imshow(holo_sum, cmap='YlGnBu_r', norm=lognorm)

ax.set_axis_off()
ax.set_xlim(760, 1400)
ax.set_ylim(835, 1475)
plt.tight_layout(pad=.1)
fig.savefig('210323_holo_diff.png', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Note down rough center - no need to be very exact just yet.

format: `[vertical axis, horizontal axis]`

In [43]:
ax.get_xlim(), ax.get_ylim()

((761.7860681881972, 1397.154730489163),
 (1455.982816376919, 835.5640049536224))

In [40]:
center = [1152, 1087]

 apply center coordinate

In [30]:
holo = holo_sum

In [31]:
def set_center(holo, center):
    n0, n1 = (2 * (s - c) for s, c in zip(holo.shape, center))
    c0, c1 = center
    holo_center = holo[c0 - n0 // 2:c0 + n0 // 2, c1 - n1 // 2:c1 + n1 // 2]
    if n0 == n1:
        return holo_center
    elif n0 > n1:
        pad0 = 0
        pad1 = (n0 - n1) // 2
    elif n0 < n1:
        pad0 = (n1 - n0) // 2
        pad1 = 0
    holo_center = np.pad(holo_center, ((pad0, pad0), (pad1, pad1)))
    return holo_center


In [32]:
holo_center = set_center(holo, center)

In [36]:
holo_center = np.pad(holo_center, 512)

In [37]:
fig, ax = plt.subplots()
ax.imshow(holo_center)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
recon = fth.reconstruct(holo_center)

Interactive centering

In [39]:
fig, ax = plt.subplots(figsize=(6, 6), constrained_layout=True)
vmin, vmax = np.percentile(recon.real, [1, 99])
ax.imshow(recon.real, vmin=vmin, vmax=vmax)

slider_c0, slider_c1 = [(p - 20, p + 20, 1) for p in center]

@widgets.interact(c0=slider_c0, c1=slider_c1)
def update(c0, c1):
    holo_center = set_center(holo, [c0, c1])
    recon = fth.reconstruct(holo_center)
    ax.imshow(recon.real, vmin=vmin, vmax=vmax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1154, description='c0', max=1174, min=1134), IntSlider(value=1080, descr…

In [40]:
fig.savefig('210322_pattersonmap.png', dpi=300)

In [9]:
center = [1154, 1080]

propagate

In [86]:
holo_center = set_center(holo_diff, center)
recon = fth.reconstruct(holo_center)

fig, ax = plt.subplots(figsize=(6, 6), constrained_layout=True)
vmin, vmax = np.percentile(recon.real, [1, 98])
ax.imshow(recon.real, vmin=vmin, vmax=vmax)

@widgets.interact_manual(distance=(-10, 10, .1))
def update(distance=(-5, 5, .1)):
    d_um = distance * 1e-6
    holo_prop = fth.propagate(holo_center.copy(), d_um, experimental_setup)
    reco_prop = fth.reconstruct(holo_prop)
    ax.imshow(reco_prop.real, vmin=vmin, vmax=vmax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='distance', max=10.0, min=-10.0), Button(description=…

In [10]:
propdist = 1.2e-6

rotate phase

In [12]:
holo_center = set_center(holo_diff, center)
holo_prop = fth.propagate(holo_center, propdist, experimental_setup)
recon = fth.reconstruct(holo_prop)

fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(6, 3.5), constrained_layout=True,
                               sharex=True, sharey=True)
vminr, vmaxr = np.percentile(recon.real, [1, 99])
vmini, vmaxi = np.percentile(recon.imag, [1, 99])
ax1.imshow(recon.real, vmin=vminr, vmax=vmaxr)
ax2.imshow(recon.imag, vmin=vmini, vmax=vmaxi)

pi = np.pi
@widgets.interact(phase=(-pi, pi, .1), flip=False)
def update(phase, flip):
    phi = phase + np.pi * flip
    recon_rot = fth.global_phase_shift(recon.copy(), phi)
    ax1.imshow(recon_rot.real, vmin=vminr, vmax=vmaxr)
    ax2.imshow(recon_rot.imag, vmin=vmini, vmax=vmaxi)

NameError: name 'set_center' is not defined

In [11]:
phase = 0.16
recon_rot = fth.global_phase_shift(recon.copy(), phase)

NameError: name 'recon' is not defined

In [102]:
min1, max1 = [int(a) for a in ax1.get_xlim()]
max0, min0 = [int(a) for a in ax1.get_ylim()]

In [104]:
min0, max0, min1, max1

(471, 731, 579, 845)

In [115]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 6))
ax.imshow(recon_rot.real[min0:max0, min1:max1], cmap='gray', vmin=-2, vmax=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
fig.savefig('210312_FB0027_C2_object.png', dpi=300)

## check number of accumulations needed

In [117]:
center, propdist, phase

([1154, 1080], 1.2e-06, 0.16)

In [118]:
center = [1154, 1080]
propdist = 1.2e-06
phase = 0.16
roi = np.s_[471:731, 579:845]

In [127]:
from mpl_toolkits.axes_grid1 import ImageGrid

In [171]:
n_accum = [1, 2, 3, 5, 10, 15, 21]

plt.ioff()

for n in n_accum:
    pos = d1.ccd.isel(index=slice(n)).mean('index').values
    neg = d2.ccd.isel(index=slice(n)).mean('index').values
    holo = pos - neg
    
    holo_center = set_center(holo, center)
    holo_prop = fth.propagate(holo_center, propdist, experimental_setup)
    
    recon = fth.reconstruct(holo_prop)
    recon_rot = fth.global_phase_shift(recon.copy(), phase).real
    
    fig = plt.figure(figsize=(4.5, 4))
    axes = ImageGrid(fig, [0, .05, .85, .85], [1, 1], cbar_mode='single')
    ax = axes[0]
    cax = axes.cbar_axes[0]
    m = ax.imshow(recon_rot[roi] / .14, vmin=-1, vmax=1, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'{n}×1s')
    cbar = plt.colorbar(m, cax=cax)
    cbar.set_ticks(np.arange(-1, 1.1, .5))
    cbar.set_label('intensity (arb. units)')
    fig.savefig(f'210312_accum_{n:02d}.png', dpi=300)
    plt.close(fig)
#     break

plt.ion()

Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um


In [168]:
fig, ax = plt.subplots()
ax.imshow(recon_rot[roi] / .12, vmin=-1, vmax=1, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [150]:
n_accum = [1, 2, 3, 5, 10, 15, 21]

plt.ioff()

for n in n_accum:
    pos = d1.ccd.isel(index=slice(n)).mean('index').values
    neg = d2.ccd.isel(index=slice(n)).mean('index').values
    holo = pos - neg
    
    holo_center = set_center(holo, center)
    holo_prop = fth.propagate(holo_center, propdist, experimental_setup)
    
    recon = fth.reconstruct(holo_prop)
    recon_rot = fth.global_phase_shift(recon.copy(), phase).real
    
    vmax = np.abs(np.percentile(recon_rot[roi], [1, 99])).min()
    
    fig, ax = plt.subplots(figsize=(4, 4), constrained_layout=True)
    m = ax.imshow(recon_rot[roi], vmin=-vmax, vmax=vmax, cmap='gray')
    ax.set_axis_off()
    ax.set_title(f'{n}×1s')
    fig.savefig(f'210312_accum_{n:02d}_anim.png', dpi=300)
    plt.close(fig)

plt.ion()

Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um
Propagation distance: 1.20um


sum reconstruction

In [174]:
pos = d1.ccd.mean('index').values
neg = d2.ccd.mean('index').values
holo = pos + neg

holo_center = set_center(holo, center)
holo_prop = fth.propagate(holo_center, propdist, experimental_setup)

recon = fth.reconstruct(holo_prop)
recon_rot = fth.global_phase_shift(recon.copy(), phase).real

fig, ax = plt.subplots(figsize=(4, 4), constrained_layout=True)
m = ax.imshow(recon_rot, cmap='gray', vmin=-.2, vmax=.2)
ax.set_axis_off()



Propagation distance: 1.20um


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## alignment sample

# Reconstruct

Reconstruct the hologramm.
1. Center the hologram.
2. Mask the beamstop.
3. Check for corsmic rays and camera defects and mask them.
4. Chose a region of interest (ROI).
5. Propagate the image and shift the phase for maximal contrast and sharpness in your ROI.

## Set center

Your hologramm is nor necessarily centered on the camera. Here you can determine the pixel coordinates of the center of the hologram and shift ist to the center of the camera image.

1. The image is plotted. Use the interactive plotting *zoom* tool (second from the right) to zoom into the image. Get the ring of the hologram centered in your field of view. You can shift the image with the *pan* tool right of *zoom*. You can use the arrows to get to the previous/next view. The *home* button (left) resets the view.
2. The limit of your FOV is read from the axes of the image. From this, the center of the hologram is determined.
3. Shift the hologram to the center.
4. The centered hologram and the original hologram are plotted next to each other to check if the centering was done correctly.

In [135]:
mi, ma = np.percentile(np.real(pos), (.01,99.99))

fig, ax = plt.subplots()
ax.imshow(np.real(pos), cmap = 'gray', vmin = mi, vmax = ma, aspect=1)
circ = plt.Circle([.5, .5], radius=.3, transform=ax.transAxes, ec='r', fc='#00000000')
ax.add_artist(circ)
ax.set_xlim(1000, 1200)
ax.set_ylim(1050, 1300)
plt.tight_layout(pad=.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
fig.savefig('149,151_holo.png', dpi=300)

In [ ]:
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()

center = [fth.integer(y1 + (y2 - y1)/2), fth.integer(x1 + (x2 - x1)/2)]

In [71]:
center

[1152, 1079]

In [103]:
center = np.array([1152, 1079])

In [ ]:
holo_c = fth.set_center(holo, center)

In [136]:
fig, ax = plt.subplots(1,2, figsize = (8, 5))
ax[0].imshow(np.real(holo), cmap = 'gray')
ax[0].set_axis_off()
ax[1].imshow(np.real(holo_c), cmap = 'gray')
ax[1].set_axis_off()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mask beamstop

Now the beamstop is masked. A circular mask is drawn with a vertain diameter and its edges are smoothed with a gauss filter.
1. Set the diameter in pixels. You can determine this by zooming into the hologram and checking the diameter of the beamstop in the hologram. Make it larger than the physical beamstop in the image so the smoothing works well.
2. The function $\mathtt{mask-beamstop(image, bs-size, sigma=10, center = None)}$ returns the masked hologram. you can set the sigma of the Gauss filter (10 is default). You can give it a center for the beamstop mask, but since you already centered the hologram, this is not necessary.
3. Plot the masked hologram to check that the beamstop is properly masked.

In [137]:
bs_diam = 35

In [138]:
holo_b = fth.mask_beamstop(holo_c, bs_diam)

In [139]:
fig, ax = plt.subplots()
ax.imshow(np.real(holo_b), cmap = 'gray')
ax.set_axis_off()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Remove Cosmic Rays

Cosmic rays are recorded as singular pixels with a very high count rate. Camera defects are also single or multiple pixels of high count rate. 

If they occur quite outside of center, they will be accounted for in the reconstruction and give high frequency modulations.
1. Look at the all the pixels in the masked hologram $>1000$.
2. Delete whole rows if necessary. Replace agglomerations of high pixels by values somewhere close.
3. Remove cosmic rays via the function $\mathtt{remove-cosmic-ray(holo, coordinates)}$. This function will replace the pixel of a single cosmic ray by the mean of its neighbors. If the cosmic ray extends on two pixels, use $\mathtt{remove-two(holo, x_coord, y_coord)}$ where one of the two coordinate variables should have a dimension of two.


In [13]:
# fig, ax = plt.subplots(figsize = (8, 8))
# ax.imshow(np.abs(holo_b)>1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# holo_b = np.delete(holo_b, 50, axis = 1)
# holo_b = np.delete(holo_b, 0, axis = 0)
# holo_b.shape

In [ ]:
# holo_b = fth.remove_cosmic_ray(holo_b, [, 822])
# holo_b = fth.remove_cosmic_ray(holo_b, [860, 935])

## Set ROI

Set a region of interest, i.e., chose which image you want to optimize.

This is done much in the same way as the center is set:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [140]:
fig, ax = fth.plot(np.real(fth.reconstruct(holo_b)), colorbar = False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi = np.array([fth.integer(x1), fth.integer(x2), fth.integer(y1), fth.integer(y2)]) #xstart, xstop, ystart, ystop

In [12]:
# roi = np.array([ 674,  850, 1121, 1292])

## Propagate and Phase Shift

Propagte your FOV and shift the phase to have all magnetic signal in the real image. This is done via ipython widget sliders (https://ipywidgets.readthedocs.io/en/latest/).

In [142]:
slider_prop, slider_phase, button = reco.propagate(holo_b, roi, experimental_setup = experimental_setup)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=10…

Button(description='Finished', style=ButtonStyle())

In [143]:
prop_dist=slider_prop.value

phase = slider_phase.value

# Save Reconstruction Data

Save the reconstructed data in a hdf5 file.

In [68]:
folder_save = '../processed/%i/'%p
# folder_save = '/home/mschndr/%i/'%p

if not(os.path.exists(folder_save)):
    print("Creating folder " + folder_save)
    os.mkdir(folder_save)

In [144]:
recon = fth.reconstruct(fth.propagate(holo_b, prop_dist*1e-6, experimental_setup = experimental_setup)*np.exp(1j*phase))[roi[2]:roi[3], roi[0]:roi[1]]

Propagation distance: 0.21um


In [145]:
fig, ax = plt.subplots(frameon = False, figsize = (recon.shape[1] / 40, recon.shape[0] / 40))
ax.imshow(np.real(recon), cmap = 'gray')
ax.set_axis_off()
ax.annotate('%04d'%p, (.015, .95), xycoords = 'axes fraction', bbox = {'alpha': .5, 'ec': None, 'fc': 'w', 'lw': None})
plt.savefig(folder_save + '2103_MaxP04_%04d.png'%p, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
reco.save_parameters(folder_save + 'P04_0620_%04d.hdf'%p, recon, factor, center, bs_diam, prop_dist, phase, roi, [p, n],
                     comment = 'beamtime reconstruction', topo = None)
# save_parameters(holo, center, prop_dist, phase, roi, folder_save, 'PETRA_P04_0620_', 'Reco1', 'RecoParam1', [p,n], bs_diam, propagate=True)